Este es un "notebook" (cuaderno simple para explorar datos lineales con métodos de descenso de gradiente con algo de dispersión (no gaussiana).

Acompaña al Capítulo 5 del libro (2 de 5).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import linear_model

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
np.random.seed(16) #establecer semilla con fines de reproducibilidad

x = np.arange(100) 

yp = 3*x + 3 + 2*(np.random.poisson(3*x+3,100)-(3*x+3)) #generar algunos datos con dispersión siguiendo la distribución de Poisson
                                                       #con valor exp = y del modelo lineal, centrado alrededor de 0

### Podemos usar nuestro conjunto de datos con valores atípicos del cuaderno anterior.


In [ ]:
np.random.seed(12) #establecer
out = np.random.choice(100,15) #seleccionar 15 índices de valores atípicos
yp_wo = np.copy(yp)
np.random.seed(12) #establecer de nuevo
yp_wo[out] = yp_wo[out] + 5*np.random.rand(15)*yp[out]

In [ ]:
plt.scatter(x,yp_wo)
plt.scatter(x,yp)

Podemos ver el efecto en el error cuadrático medio


In [ ]:
model.fit(x.reshape(-1,1),yp_wo)

pendiente, intercepto  = model.coef_, model.intercept_

print(pendiente, intercepto)

### Registro de aprendizaje
    
¿Cuál es la principal diferencia en la pendiente y el intercepto que encontraste, en comparación con el caso de que no haya valores atípicos?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
La pendiente cambia notablemente, de ~3 a ~4, porque los valores atípicos afectan en gran medida el error cuadrático medio. El intercepto también cambia significativamente (pero recuerda que el intercepto es mucho más difícil de identificar en un problema lineal).
```
    
</p>
</details>

### Ahora implementemos la forma más simple de descenso de gradiente: lote, estocástico y mini lote, uno por uno.


In [ ]:
X = np.c_[np.ones((100, 1)), x] # agregue x0 = 1 a cada instancia; este es el término de sesgo

print(X.shape) #la forma (shape en inglés) es número de instancias x número de parámetros

In [ ]:
theta_ne = np.array([[1.548],[3.978]])

In [ ]:
loss_ne = np.mean((X.dot(theta_ne) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_ne

### Descenso de gradiente por lote. En inglés Batch Gradient Descent (GD).

In [ ]:
np.random.seed(10) #Mismas condiciones iniciales para todos.

eta = 0.0001
n_iterations = 1000 #¡Intenta cambiar el  número de iteraciones !
m = 100

theta_path_bgd = []

theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X.T.dot(X.dot(theta) - yp_wo.reshape(-1,1))
    theta = theta - eta * gradients
    theta_path_bgd.append(theta)

theta_path_bgd = np.array(theta_path_bgd) #guardar el camino

theta_bgd = theta #resultado final

In [ ]:
theta_bgd

In [ ]:
loss_bgd = np.sum(1/m*(X.dot(theta_bgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_bgd

### Registro de aprendizaje

¿Cuál es la diferencia porcentual entre el valor final de la pérdida encontrada por el Descenso de gradiente por lote y por la ecuación normal?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
(loss_ne-loss_bgd)/loss_ne*100

(debe ser del orden de 10^-5, mostrando que descenso de gradiente por lote y la ecuación normal son esencialmente equivalentes.)
```
    
</p>
</details>

¿Qué sucede con esta comparación si aumenta el número de iteraciones en el descenso de gradiente por lote?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Deberían acercarse.
```

### Descenso de Gradientes Estocástico. En inglés Stochastic gradient descent  (SGD)

In [ ]:
np.random.seed(10) #Mismas condiciones iniciales para todos.

theta = np.random.randn(2,1) 

eta = 0.000005

n_iterations = 10000 #mas iteraciones

theta_path_sgd = []

for epoch in range(n_iterations):
    
        random_index = np.random.randint(m) # elegir un ejemplo de los datos
        
        x_one = X[random_index:random_index+1]
        
        y_one = yp_wo[random_index:random_index+1]
        
        gradients = 2 * x_one.T.dot(x_one.dot(theta) - y_one)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 

theta_path_sgd = np.array(theta_path_sgd)

theta_sgd = theta

In [ ]:
theta_sgd

Una vez más, encontramos una theta similar, pero no exactamente la misma.

In [ ]:
loss_sgd = np.sum(1/m*(X.dot(theta_sgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_sgd

In [ ]:
(loss_ne-loss_sgd)/loss_sgd*100 #diferencia porcentual con la ecuación normal

### Registro de aprendizaje

¿Debería preocuparnos que el valor final de la pérdida para el descenso de gradientes estocástico no sea tan cercano al encontrado por la ecuación Normal?
    

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
No, porque sabemos que las fluctuaciones estadísticas de los algoritmos de descenso de gradientes estocástico son grandes y no se garantiza que la pérdida disminuya en cada paso.
```

### Descenso de gradiente de mini-lote  .En inglés Mini batch (MGD).

In [ ]:
# Consulte también las notas de implementación aquí (notas en inglés): https://sebastianraschka.com/faq/docs/sgd-methods.html

np.random.seed(10)

theta = np.random.randn(2,1) 

eta = 0.000005

n_iterations = 1000

theta_path_mgd = []

minibatch_size = 10 #tamaño del mini-lote

for epoch in range(n_iterations):
    
    shuffled_indices = np.random.permutation(m) #Desordenar la matriz aleatoriamente.
    
    X_shuffled = X[shuffled_indices]
    
    y_shuffled = yp_wo.reshape(-1,1)[shuffled_indices]
    
    xi = X_shuffled[:minibatch_size]
    
    yi = y_shuffled[:minibatch_size]
    
    gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
    
    theta = theta - eta * gradients
    
    theta_path_mgd.append(theta)

theta_path_mgd = np.array(theta_path_mgd)

theta_mgd = theta 

print(theta_mgd)

In [ ]:
loss_mgd = np.sum(1/m*(X.dot(theta_mgd) - yp_wo.reshape(-1,1))**2)

In [ ]:
loss_mgd

In [ ]:
(loss_ne-loss_mgd)/loss_ne*100 #diferencia porcentual con la ecuación normal

Lo mismo que antes.

Es muy interesante observar el camino tomado por el descenso de gradiente en los tres casos. Los colores cada vez más oscuros denotan pasos posteriores.

In [ ]:
plt.figure(figsize=(14,8))

plt.scatter(theta_path_sgd[::10, 0].flatten(), theta_path_sgd[::10, 1].flatten(), marker = 's', s = 5, \
         label="DG Estocástico, N$_{it}$ = 10000", c = np.arange(1000), cmap=plt.cm.Purples)
plt.scatter(theta_path_mgd[:, 0].flatten(), theta_path_mgd[:, 1].flatten(), marker = "+", s = 12, linewidth=1, \
            label="DG Mini-lote, N$_{it}$ = 1000", c = np.arange(1000), cmap=plt.cm.Greens)
plt.scatter(theta_path_bgd[:, 0].flatten(), theta_path_bgd[:, 1].flatten(), marker = "d", s = 12, linewidth=1, \
            label="DG Lote, N$_{it}$ = 1000", c = np.arange(1000,0,-1), cmap=plt.cm.copper)

plt.scatter(theta_sgd[0],theta_sgd[1], marker = "s", s = 100, color = 'Purple', alpha = 0.5)
plt.scatter(theta_mgd[0],theta_mgd[1], marker = "+", s = 200, color = 'DarkGreen', alpha = 1)
plt.scatter(theta_bgd[0],theta_bgd[1], marker = "d", s = 100, color = 'k', alpha = 0.5)
#plt.text(1.5,3.978,'Normal Equation solution X')

legend = plt.legend(loc="upper left", fontsize=16)


for i in range(3):

    legend.legendHandles[i].set_color('k')
    legend.legendHandles[i]._sizes = [30]

plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20)

plt.axis([1.3, 1.4, 2.5, 6.5])

#plt.savefig('AllThePaths.png', dpi = 300)
plt.show()


### Sin valores atípicos

In [ ]:
#DG por Lotes

np.random.seed(10) #Mismas condiciones iniciales para todos.

eta = 0.0001
n_iterations = 1000
m = 100

theta_path_bgd = []

theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X.T.dot(X.dot(theta) - yp.reshape(-1,1))
    theta = theta - eta * gradients
    theta_path_bgd.append(theta)

theta_path_bgd = np.array(theta_path_bgd)

#DG Estocástico:

np.random.seed(10) #same initial conditions for all

theta = np.random.randn(2,1) 

eta = 0.00005

n_iterations = 1000

theta_path_sgd = []

for epoch in range(n_iterations):
    
        random_index = np.random.randint(m) # pick one example from the data 
        xi = X[random_index:random_index+1]
        yi = yp[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

theta_path_sgd = np.array(theta_path_sgd)


#DG por mini-lote:

np.random.seed(10)

theta = np.random.randn(2,1) 

eta = 0.0001

n_iterations = 1000

theta_path_mgd = []

minibatch_size = 10

for epoch in range(n_iterations):
    
    shuffled_indices = np.random.permutation(m) #Desordenar la matriz aleatoriamente.
    X_shuffled = X[shuffled_indices]
    y_shuffled = yp.reshape(-1,1)[shuffled_indices]
    
    xi = X_shuffled[0:minibatch_size] #sin reemplazo, técnicamente deberíamos
    yi = y_shuffled[0:minibatch_size]
    gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
    theta = theta - eta * gradients
    theta_path_mgd.append(theta)

theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(14,8))
plt.scatter(theta_path_bgd[:, 0].flatten(), theta_path_bgd[:, 1].flatten(), marker = "d", s = 12, linewidth=1, \
            label="Lote", c = np.arange(1000,0,-1), cmap=plt.cm.copper)
plt.scatter(theta_path_sgd[:, 0].flatten(), theta_path_sgd[:, 1].flatten(), marker = 's', s = 5, \
         label="Estocástico", c = np.arange(1000), cmap=plt.cm.Purples)
plt.scatter(theta_path_mgd[:, 0].flatten(), theta_path_mgd[:, 1].flatten(), marker = "+", s = 12, linewidth=1, \
            label="Mini-lote", c = np.arange(1000), cmap=plt.cm.Greens)
legend = plt.legend(loc="upper left", fontsize=16)

for i in range(3):

    legend.legendHandles[i].set_color('k')
    legend.legendHandles[i]._sizes = [30]

plt.show()

Ejercicio 1: tenga en cuenta lo que sucede con tasas de aprendizaje más grandes y tasas de aprendizaje más pequeñas. ¿Sería una solución una tasa de aprendizaje adaptativa? Cualitativamente, ¿cómo lo elegirías?

Ejercicio 2: Examine los gradientes para descubrir por qué el descenso de gradiente por lote deja de actualizar la pendiente con bastante rapidez. ¿Sería esto una preocupación en términos de quedarse atascado en mínimos locales (en funciones de pérdida que no son convexas)?
